### Квантует DeepLabV3 MobilenetV3

Стартуем с трейнлупа, который нам выдали pytorch

Датасет COCO, https://cocodataset.org/#download 
Качаем train2017 и val2017

Можно использовать [сабсет](https://drive.google.com/file/d/1qdtAbK-iOsgJZxjbBva0pw2Vi5penjPc/view?usp=sharing) трейна на 20000, но тогда заранее залезте в класс датасета, и добавте работу с пропущенными картинками

Баллы: 20 баллов Static Quantization + 20 баллов Quantization Aware Training + 10 баллов Distillation

In [3]:
!pip3 install pycocotools

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 49.3 MB/s eta 0:00:00


In [1]:
import datetime
import os
import pickle
import time
from copy import deepcopy
from pathlib import Path

import torch
import torch.utils.data
from torch import nn
from torch.ao.quantization.quantize_fx import convert_fx
from torch.ao.quantization.quantize_fx import fuse_fx
from torch.optim.lr_scheduler import PolynomialLR
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from tqdm import tqdm

import utils
from quantization_utils.fake_quantization import fake_quantization
from quantization_utils.static_quantization import quantize_static
from train import evaluate
from train import get_dataset
from train import train_one_epoch

In [2]:
# Вытащил дефолтные аргументы, чтобы не упражняться с argparse в ноутбуке
with Path('./torch_default_args.pickle').open('rb') as file:
    args = pickle.load(file)

In [3]:
# Подобирайте под ваше железо
args.data_path = '/data/coco/'
args.epochs = 1
args.batch_size = 16
args.workers = 16

In [4]:
args

Namespace(amp=False, aux_loss=False, backend='pil', batch_size=16, data_path='/data/coco/', dataset='coco', device='cuda', dist_url='env://', epochs=1, lr=0.01, lr_warmup_decay=0.01, lr_warmup_epochs=0, lr_warmup_method='linear', model='deeplabv3_mobilenet_v3_large', momentum=0.9, output_dir='.', print_freq=10, resume='', start_epoch=0, test_only=False, use_deterministic_algorithms=False, use_v2=False, weight_decay=0.0001, weights=None, weights_backbone=None, workers=16, world_size=1)

### Сначала просто валидация обычной сетки, прям на гпу

In [5]:
model = deeplabv3_resnet50(weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1)
model.eval();

In [6]:
if args.output_dir:
    utils.mkdir(args.output_dir)

utils.init_distributed_mode(args)

device = torch.device(args.device)

dataset_test, num_classes = get_dataset(args, is_train=False)

test_sampler = torch.utils.data.SequentialSampler(dataset_test)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=16, sampler=test_sampler, num_workers=args.workers, collate_fn=utils.collate_fn
)

Not using distributed mode
loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [ ]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
model.cuda()
confmat = evaluate(model, data_loader_test, device=device, num_classes=num_classes)
print(confmat)

Test:  [  0/313]  eta: 0:33:29    time: 6.4195  data: 2.9631  max mem: 6087
Test:  [ 10/313]  eta: 0:11:26    time: 2.2645  data: 0.2985  max mem: 6550
Test:  [ 20/313]  eta: 0:10:04    time: 1.8443  data: 0.0317  max mem: 6746
Test:  [ 30/313]  eta: 0:10:15    time: 2.1259  data: 0.0345  max mem: 9602
Test:  [ 40/313]  eta: 0:09:48    time: 2.2516  data: 0.0361  max mem: 10826
Test:  [ 50/313]  eta: 0:09:17    time: 2.0346  data: 0.0329  max mem: 10826
Test:  [ 60/313]  eta: 0:08:52    time: 2.0021  data: 0.0326  max mem: 10826
Test:  [ 70/313]  eta: 0:08:28    time: 2.0223  data: 0.0335  max mem: 10826
Test:  [ 80/313]  eta: 0:08:07    time: 2.0563  data: 0.0344  max mem: 10826
Test:  [ 90/313]  eta: 0:07:41    time: 1.9829  data: 0.0346  max mem: 10826
Test:  [100/313]  eta: 0:07:15    time: 1.8560  data: 0.0322  max mem: 10826
Test:  [110/313]  eta: 0:06:56    time: 1.9832  data: 0.0342  max mem: 10826
Test:  [120/313]  eta: 0:06:38    time: 2.1665  data: 0.0368  max mem: 12239
Tes

### Заквантуем статические сетку, посмотрим на точность и скорость

In [10]:
# Квантуем
# Делаем fuse, делаем quantize_static и quantize_utils (посмотрите что там с кодом)
# Можно покрутить параметр num_batches, чтобы посмотреть сколько нужно данных на калибровку
q_model = quantize_static(
    model=model,
    data_loader=data_loader_test,
    num_batches=16,
)
fused_model = fuse_fx(q_model)

/home/hflabs/anaconda3/envs/ds/lib/python3.8/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [ ]:
# Замерим скорость квантованной модели на CPU
# Не забываем, от размера батча будет зависить буст

In [ ]:
# Замеряем с бачтом 1, буста нет (3 batch/s)
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, sampler=test_sampler, num_workers=args.workers, collate_fn=utils.collate_fn
)
confmat_1_batch = evaluate(fused_model, data_loader_test, device='cpu', num_classes=num_classes)
print(confmat_1_batch)
# Замеряем с батчом 32, буст есть (
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=16, sampler=test_sampler, num_workers=args.workers, collate_fn=utils.collate_fn
)
confmat_32_batch = evaluate(fused_model, data_loader_test, device='cpu', num_classes=num_classes)
print(confmat_32_batch)
# Мораль, latency != throughput. В сетке всегда есть накладные расходы, кроме перемалывания матричек

In [ ]:
# Замерим скорость оригинальной модели на CPU
confmat_original_batch = evaluate(model, data_loader_test, device='cpu', num_classes=num_classes)
print(confmat_original_batch)
# У меня на intel i9 при батчсайзе 32 получился x2 буст

In [ ]:
# Посчитаем метрики квантованной модели
# У меня была просадка где-то до 65 IoU
q_model.cpu()
confmat = evaluate(q_model, data_loader_test, device='cpu', num_classes=num_classes)
print(confmat)

### Делаем Quantization Aware Training. Используем готовый трейнплуп от pytorch

In [ ]:
# Делаем фейк квантование, идём смотреть quantization_utils

### Тут берём из train.py скрипт main() и вытаскиваем трейн луп

1. Не забыть провалидировать модель fake quant до qat
2. Не забыть провалидировать модель после обучения
3. Конвертировать модель из fake quant в обычный quant
4. Проверить точность и скорость модели
Должно хватить пары эпох, lr надо будет покрутить.
Цель минимум это поднять точность конвертированной QAT модели (будет у вас QAT до обучения 55, станет 56, молодцы!)

In [ ]:
# Подсказка, учить надо в train и на GPU
qat_model.cuda();

In [ ]:
# Инференс делаем на cpu, предварительно конвертируя модельку на CPU
qat_model.cpu()
int_qat_model = convert_fx(qat_model)

In [ ]:
# Точность модели fake quant и квантованной после конвертации будут разные
# Так и должно быть, всё таки мы эмулировали квантование.

### Балуемся с дистилляцией
Врываемся в train.py и добавляем туда дистилляцию, просто по последнему слою (до софтмакса, на логитах) делаем стягивание по MSE

Цель поднять точность и ускорить сходимость.

Балуемся с весами обычного и distill лосса.

Можно вообще выкинуть classification loss и смоделировать ситуацию когда вам не выдали лейблов (жиза)